# Principio de Convolución y Ejemplo Interactivo de Suavizado

## ¿Qué es la Convolución Discreta?

La convolución es una operación matemática fundamental en el Procesamiento Digital de Señales (DSP). Describe cómo un sistema Lineal e Invariante en el Tiempo (LTI) modifica una señal de entrada para producir una señal de salida.

Si tenemos:
* $x(n)$: La secuencia de entrada (por ejemplo, una señal medida).
* $h(n)$: La respuesta al impulso del sistema LTI (define cómo el sistema reacciona a una entrada puntual).

La salida $y(n)$ del sistema se calcula mediante la **suma de convolución**:

$$ y(n) = (x * h)[n] = \sum_{m=-\infty}^{\infty} x(m)h(n-m) = \sum_{m=-\infty}^{\infty} h(m)x(n-m) $$

**Interpretación:**
La salida $y(n)$ en un instante $n$ es una **suma ponderada** de las muestras de entrada $x$. La segunda forma de la suma, $\sum h(m)x(n-m)$, lo muestra claramente: $h(m)$ actúa como los "pesos" aplicados a las muestras de entrada presentes y pasadas $x(n-m)$. Se puede visualizar como "invertir" la secuencia $h(m)$ y deslizarla sobre $x(m)$, calculando el producto punto en cada posición $n$.

In [1]:
# --------------- Imports ---------------
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import FloatSlider, VBox, interactive_output, Layout

# --- Datos de Ejemplo (Basado en Clase 10 y Orfanidis Ejemplo 4.1.1) ---
x_signal = np.array([1, 1, 2, 1, 2, 2, 1, 1])
L = len(x_signal)
initial_h = np.array([1.0, 2.0, -1.0, 1.0]) # h inicial
M = len(initial_h) - 1
n_x = np.arange(L)
n_h_base = np.arange(M + 1) # Eje n para h (no cambia)

# --- Sliders para los coeficientes de h(n) ---
style = {'description_width': 'initial'}
layout_slider = Layout(width='80%') # Ajustar ancho de sliders
h0_slider = FloatSlider(min=-2, max=3, step=0.1, value=initial_h[0], description='h0', style=style, layout=layout_slider)
h1_slider = FloatSlider(min=-2, max=3, step=0.1, value=initial_h[1], description='h1', style=style, layout=layout_slider)
h2_slider = FloatSlider(min=-2, max=3, step=0.1, value=initial_h[2], description='h2', style=style, layout=layout_slider)
h3_slider = FloatSlider(min=-2, max=3, step=0.1, value=initial_h[3], description='h3', style=style, layout=layout_slider)

# Agrupar sliders
controls = VBox([h0_slider, h1_slider, h2_slider, h3_slider])

# --- Contenedor para el gráfico ---
plot_output = widgets.Output()

# --- Función de Actualización y Creación de Gráfico ---
def update_plot_and_show(h0, h1, h2, h3):
    """Calcula convolución y crea/muestra un nuevo gráfico Plotly."""
    h_interactive = np.array([h0, h1, h2, h3])
    y_interactive = np.convolve(h_interactive, x_signal)
    Ly_interactive = L + M
    n_y_interactive = np.arange(Ly_interactive)

    # Crear una NUEVA figura cada vez
    fig_new = make_subplots(rows=3, cols=1,
                            shared_xaxes=False,
                            vertical_spacing=0.1,
                            subplot_titles=('Señal de Entrada Fija x(n)',
                                            f'h(n)=[{h0:.1f}, {h1:.1f}, {h2:.1f}, {h3:.1f}]', # Título dinámico para h(n)
                                            'Salida por Convolución y(n) = x(n) * h(n)')
                           )

    # 1. Gráfico x(n)
    fig_new.add_trace(go.Scatter(x=n_x, y=x_signal, mode='markers', name='x(n)', marker=dict(color='blue', size=8)), row=1, col=1)
    for i in range(L):
        fig_new.add_shape(type='line', x0=n_x[i], y0=0, x1=n_x[i], y1=x_signal[i], line=dict(color='blue', width=1), row=1, col=1)

    # 2. Gráfico h(n)
    fig_new.add_trace(go.Scatter(x=n_h_base, y=h_interactive, mode='markers', name='h(n)', marker=dict(color='red', size=8)), row=2, col=1)
    for i in range(M + 1):
        fig_new.add_shape(type='line', x0=n_h_base[i], y0=0, x1=n_h_base[i], y1=h_interactive[i], line=dict(color='red', width=1), row=2, col=1)

    # 3. Gráfico y(n)
    fig_new.add_trace(go.Scatter(x=n_y_interactive, y=y_interactive, mode='markers', name='y(n)', marker=dict(color='green', size=8)), row=3, col=1)
    for i in range(Ly_interactive):
        fig_new.add_shape(type='line', x0=n_y_interactive[i], y0=0, x1=n_y_interactive[i], y1=y_interactive[i], line=dict(color='green', width=1), row=3, col=1)

    # Layout y ejes
    fig_new.update_layout(height=700, showlegend=False, title_text="Convolución Interactiva FIR")
    fig_new.update_xaxes(title_text="n", range=[-1, L], row=1, col=1)
    fig_new.update_yaxes(title_text="x(n)", row=1, col=1)
    fig_new.update_xaxes(title_text="n", range=[-1, M+1], row=2, col=1)
    fig_new.update_yaxes(title_text="h(n)", range=[-3, 4], row=2, col=1) # Ajustar si es necesario
    fig_new.update_xaxes(title_text="n", range=[-1, Ly_interactive], row=3, col=1)
    min_val_y = min(y_interactive) if len(y_interactive) > 0 else -1
    max_val_y = max(y_interactive) if len(y_interactive) > 0 else 1
    fig_new.update_yaxes(title_text="y(n)", range=[min_val_y - abs(min_val_y)*0.1 - 1, max_val_y + abs(max_val_y)*0.1 + 1], row=3, col=1)

    # Limpiar el output anterior y mostrar el nuevo gráfico
    with plot_output:
        plot_output.clear_output(wait=True) 
        fig_new.show() 
        
# --- Conectar Sliders a la Función de Actualización usando interactive_output ---
out_plot = interactive_output(update_plot_and_show, {
    'h0': h0_slider, 'h1': h1_slider, 'h2': h2_slider, 'h3': h3_slider
})

# --- Mostrar Controles y Gráfico ---
print("Ajusta los sliders para ver cómo cambia la convolución:")
# Mostrar los controles y el contenedor de salida del gráfico
display(VBox([controls, plot_output]))

# Llamar una vez manualmente para mostrar el gráfico inicial
update_plot_and_show(h0_slider.value, h1_slider.value, h2_slider.value, h3_slider.value)

print("\n--- Fin del Código ---")

Ajusta los sliders para ver cómo cambia la convolución:



--- Fin del Código ---


## Ejemplo Práctico: Filtro Promediador FIR para Suavizar Ruido

Un uso muy común de la convolución es el **suavizado de señales ruidosas** mediante un filtro de **promedio móvil (Moving Average)**. Este es un filtro FIR simple.

**Idea:** Si una señal tiene ruido aleatorio de alta frecuencia superpuesto a una tendencia más lenta, promediar varios puntos consecutivos debería reducir el impacto del ruido (que tiende a cancelarse) y resaltar la tendencia.

**Filtro Promediador de N puntos:**
La respuesta al impulso de este filtro tiene longitud $N$ y todos sus coeficientes son iguales a $1/N$:
$$ h(n) = \begin{cases} 1/N & \text{si } 0 \le n < N \\ 0 & \text{en otro caso} \end{cases} $$

**Convolución:**
La salida $y(n)$ se calcula convolucionando la entrada ruidosa $x(n)$ con este filtro $h(n)$:
$$ y(n) = \sum_{m=0}^{N-1} h(m)x(n-m) = \sum_{m=0}^{N-1} \frac{1}{N} x(n-m) = \frac{1}{N} \sum_{m=0}^{N-1} x(n-m) $$
$$ y(n) = \frac{x(n) + x(n-1) + \dots + x(n-N+1)}{N} $$
La salida es simplemente el promedio de las últimas N muestras de entrada

**Visualización Interactiva (Referencia al Código Python):**
El código Python te permite ajustar la longitud $N$ del filtro promediador y observar interactivamente:
1.  La señal original ruidosa $x(n)$.
2.  La respuesta al impulso $h(n)$ del filtro (cambia con $N$).
3.  La señal de salida suavizada $y(n)$ (resultado de la convolución). Se superpone la señal escalón ideal para comparación.

In [2]:
# --------------- Imports ---------------
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import IntSlider, Dropdown, VBox, HBox, interactive_output, Layout
import math # Para pi

# --- Funciones para Generar Señales de Ejemplo ---
L_signal = 300 # Longitud de la señal de entrada
n_signal = np.arange(L_signal)
np.random.seed(420) # reproducibilidad

def generate_noisy_step():
    s_ideal = np.zeros(L_signal)
    s_ideal[L_signal // 4 : 3 * L_signal // 4] = 5.0
    noise = np.random.normal(0, 0.8, L_signal) # Ruido 0.8
    x_signal = s_ideal + noise
    return x_signal, s_ideal, "Escalón + Ruido"

def generate_noisy_sine():
    f0 = 5.0 # Frecuencia de la sinusoide 
    fs_sim = L_signal # Frecuencia de muestreo simulada para que quepan ciclos
    s_ideal = 4.0 * np.sin(2 * math.pi * f0 * n_signal / fs_sim)
    noise = np.random.normal(0, 0.8, L_signal) # Ruido 0.8
    x_signal = s_ideal + noise
    return x_signal, s_ideal, "Sinusoide + Ruido"

def generate_noisy_ppg():
    # Modelo PPG simplificado 
    period = 25 # Período del pulso simulado
    s_ideal = np.zeros(L_signal)
    for i in range(0, L_signal, period):
        peak_width = 5
        if i + peak_width < L_signal:
             # Pico principal 
             s_ideal[i : i + peak_width] = np.linspace(0, 4, peak_width)
             if i + peak_width*2 < L_signal:
                  s_ideal[i + peak_width : i + peak_width*2] = np.linspace(4, 0, peak_width)
             # Muesca dicrótica simple (pequeña caída)
             if i + peak_width*2 + 3 < L_signal:
                  s_ideal[i + peak_width*2 + 1] = -0.5
                  s_ideal[i + peak_width*2 + 2] = -0.5


    noise = np.random.normal(0, 0.8, L_signal) # Ruido 0.8
    x_signal = s_ideal + noise
    return x_signal, s_ideal, "PPG Simulado + Ruido"

# --- Widgets de Control ---
style = {'description_width': 'initial'}
layout_widget = Layout(width='80%')

# Slider para Longitud del Filtro
N_slider = IntSlider(min=1, max=25, step=1, value=5,
                     description='Longitud Filtro (N)', style=style, layout=layout_widget)

# Dropdown para Tipo de Señal
signal_options = ['Escalón Ruidoso', 'Sinusoide Ruidosa', 'PPG Simulado Ruidoso']
signal_dropdown = Dropdown(options=signal_options, value=signal_options[0],
                           description='Tipo de Señal:', style=style, layout=layout_widget)


# --- Contenedor para el gráfico ---
plot_output_smooth = widgets.Output()

# --- Función de Actualización y Creación de Gráfico ---
def update_smoothing_plot(N, signal_type):
    """Calcula convolución y crea/muestra gráfico para la señal seleccionada."""
    # 1. Seleccionar/Generar la señal de entrada adecuada
    if signal_type == 'Escalón Ruidoso':
        x_signal, s_ideal, title_x = generate_noisy_step()
    elif signal_type == 'Sinusoide Ruidosa':
        x_signal, s_ideal, title_x = generate_noisy_sine()
    elif signal_type == 'PPG Simulado Ruidoso':
        x_signal, s_ideal, title_x = generate_noisy_ppg()
    else: # Fallback
        x_signal, s_ideal, title_x = generate_noisy_step()

    # 2. Crear el filtro promediador h(n)
    h_filter = np.ones(N) / N
    M = N - 1 # Orden del filtro

    # 3. Calcular la convolución
    y_smoothed = np.convolve(h_filter, x_signal)
    Ly_smoothed = L_signal + M
    n_y_smoothed = np.arange(Ly_smoothed)

    # 4. Crear una NUEVA figura
    fig_smooth = make_subplots(rows=3, cols=1,
                               shared_xaxes=False,
                               vertical_spacing=0.15,
                               subplot_titles=(f'Original: {title_x}',
                                               f'Filtro Promediador h(n) (N={N})',
                                               'Señal Suavizada y(n) = x(n) * h(n)')
                              )

    # Gráfico x(n)
    n_x_plot = np.arange(len(x_signal)) # Usar longitud real de x_signal
    fig_smooth.add_trace(go.Scatter(x=n_x_plot, y=x_signal, mode='lines+markers', name='x(n)',
                                     marker=dict(color='blue', size=4), line=dict(color='lightblue', width=1)),
                         row=1, col=1)

    # Gráfico h(n)
    n_h_filter = np.arange(N)
    fig_smooth.add_trace(go.Scatter(x=n_h_filter, y=h_filter, mode='markers', name='h(n)',
                                     marker=dict(color='red', size=8)),
                         row=2, col=1)
    for i in range(N):
        fig_smooth.add_shape(type='line', x0=n_h_filter[i], y0=0, x1=n_h_filter[i], y1=h_filter[i],
                             line=dict(color='red', width=1), row=2, col=1)

    # Gráfico y(n)
    fig_smooth.add_trace(go.Scatter(x=n_y_smoothed, y=y_smoothed, mode='lines+markers', name='y(n)',
                                     marker=dict(color='green', size=4), line=dict(color='lightgreen', width=1)),
                         row=3, col=1)
    # También graficar la señal ideal s_ideal si existe
    if s_ideal is not None:
         n_s_plot = np.arange(len(s_ideal)) # Usar longitud real de s_ideal
         fig_smooth.add_trace(go.Scatter(x=n_s_plot, y=s_ideal, mode='lines', name='Ideal s(n)',
                                      line=dict(color='orange', width=2, dash='dash')),
                          row=3, col=1)

    # Layout y ejes
    fig_smooth.update_layout(height=700, showlegend=False, title_text="Suavizado Interactivo (Convolución)")
    fig_smooth.update_xaxes(title_text="n (muestras)", row=1, col=1)
    fig_smooth.update_yaxes(title_text="Amplitud", row=1, col=1)
    fig_smooth.update_xaxes(title_text="n (coeficientes)", range=[-1, N], row=2, col=1)
    min_h_val = 0
    max_h_val = 1.1 / N_slider.min if N_slider.min > 0 else 1.1
    fig_smooth.update_yaxes(title_text="h(n)", range=[min_h_val - 0.1, max_h_val], row=2, col=1)
    fig_smooth.update_xaxes(title_text="n (muestras)", row=3, col=1)
    fig_smooth.update_yaxes(title_text="Amplitud", row=3, col=1)

    # Limpiar y mostrar
    with plot_output_smooth:
        plot_output_smooth.clear_output(wait=True)
        fig_smooth.show()

# --- Conectar Widgets y Mostrar ---
# Conectar tanto el slider como el dropdown a la función
out_smooth = interactive_output(update_smoothing_plot, {
    'N': N_slider,
    'signal_type': signal_dropdown
})

print("Selecciona el tipo de señal y ajusta la longitud (N) del filtro:")
# Mostrar los controles
controls_box = VBox([signal_dropdown, N_slider])
display(VBox([controls_box, plot_output_smooth]))

# Mostrar gráfico inicial con los valores por defecto
update_smoothing_plot(N_slider.value, signal_dropdown.value)

print("\n--- Fin del Código ---")

Selecciona el tipo de señal y ajusta la longitud (N) del filtro:



--- Fin del Código ---
